In [1]:
#libraries
import re
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import pysal.model.spreg as spreg
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pysal.lib
import geopandas as gpd
import pandas as pd
from libpysal.weights import DistanceBand
from scipy.spatial import distance_matrix
import pandas as pd
from scipy.stats import pearsonr
from esda.moran import Moran
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import spreg
from spreg import SURlagIV
from spreg import GM_Lag
import libpysal
from spreg import AKtest
from spreg import ML_Lag
from spreg import GM_Combo

**Loading Path**

In [2]:
# Dependent variable
dep_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\2.2 Saving the Dependent Variables"

# Independent variable
ind_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\1.2 Saving the Independent Variables"

# Steamengines and horsepower
power_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data"

In [3]:
# lets read the metadata
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\control.dta") as reader:
    meta_control = reader.variable_labels()
    
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\_FINAL_c_steamenginges_1904.dta") as reader:
    meta_steam = reader.variable_labels()

In [4]:
meta_control

{'id_inequality': 'County ID',
 'rb_id': 'District ID',
 'rb_name': 'District name',
 'province_id': 'Province ID',
 's_industry': 'Share industry employment',
 's_prot': 'Share Protestants',
 's_german': 'Share German-speaking',
 'longitud': 'Longitude',
 'latitud': 'Latitude',
 'slope_km': 'Terrain slope',
 'soil_suitability': 'Soil suitability',
 'mean_temp': 'Average temperature',
 'mean_preci': 'Average precipitation',
 'dist_100km': 'Distance to the nearest river',
 'log_distance_carbon_c': 'Least distance to carbon field (in log)'}

In [5]:
# Load Skill levels per Landkreis
skills_df = pd.read_csv(f"{dep_var_path}\Main_Dep.csv",
                        sep=",")

# Load control variables
control_df = pd.read_csv(f"{ind_var_path}\Independent Variables.csv",
                         sep=",")

# Load SteamEngine and Horsepower data
power_df = pd.read_stata(f"{power_path}\_FINAL_c_steamenginges_1904.dta")

In [6]:
# Merge the dataframes using an inner join on the "id_inequality" column
prep_df = pd.merge(control_df, skills_df, on="id_inequality", how="inner")
merged_df = pd.merge(prep_df, power_df, on="id_inequality", how="inner")

In [7]:
# Transform steamengines to steamengines per worker
merged_df["steamengines_number"] = (merged_df["steamengines_number"] / merged_df["d_Hauptberuf"])*1000
merged_df["horsepower"] = (merged_df["horsepower"] / merged_df["d_Hauptberuf"])*1000

merged_df["steamengines_number"] = merged_df["steamengines_number"]
merged_df["horsepower"] = np.log(merged_df["horsepower"])

In [8]:
# Drop Erblanden
#district_Erblanden = [31]
#merged_df = merged_df[~merged_df["rb_id"].isin(district_Erblanden)]

In [9]:
merged_df["per_unskilled"] = merged_df['per_unskilled']*100
merged_df["per_low"] = merged_df['per_low']*100
merged_df["per_medium"] = merged_df['per_medium']*100
merged_df["per_high"] = merged_df['per_high']*100

# 2SLS
Lets run a good'ol reliable IV first

## 2SLS function
I build a function that runs multiple 2SLS in a row, each time adding another variable from a list of control variables

In [10]:
def twoSLS(df, IV, first_stage_dependent, controls, second_stage_independent):
    """
    Performs a series of Two-Stage Least Squares (2SLS) regressions on a given DataFrame.
    In each iteration, an additional control variable from the provided list is included.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    controls (list of str): A list of control variable names. Each iteration includes an additional control variable.
    second_stage_independent (str): The name of the main independent variable for the second stage regression.

    Returns:
    tuple of DataFrames: The first DataFrame contains the results (coefficients, significance, 
                         Adj. R-squared, F-statistic) of each first stage regression. 
                         The second DataFrame contains the results 
                         of each second stage regression. Significance is denoted by stars: 
                         *** for p < 0.01, ** for p < 0.05, and * for p < 0.1.
                         The third DataFrame contains the residuals.
    """

    first_stage_results = []
    second_stage_results = []
    # For analysis of spatial autocorrelation
    residuals = []

    for i in range(len(controls) + 1):
        # Subset of control variables for this iteration
        subset_controls = controls[:i]

        # Build the formula, handling the case with no controls
        controls_formula_part = " + ".join(subset_controls)
        if controls_formula_part:
            controls_formula_part = " + " + controls_formula_part

        # To avoid SettingWithCopyWarning
        df_copy = df.copy()

        # First Stage Regression
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()

        # Add the predicted values as a new column in the copy
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression
        second_stage_formula = f"{second_stage_independent} ~ predicted{controls_formula_part}"
        second_stage_model = smf.ols(second_stage_formula, data=df_copy)
        second_stage_fit = second_stage_model.fit()

        def format_coef(coef, std_err, p_value):
            stars = ""
            if p_value < 0.01:
                stars = "***"
            elif p_value < 0.05:
                stars = "**"
            elif p_value < 0.1:
                stars = "*"
    
            # Check if coef or std_err is smaller than 0.000 and format accordingly
            coef_str = f"{coef:.10e}" if abs(coef) < 0.000 else f"{coef:.3f}"
            std_err_str = f"{std_err:.10e}" if abs(std_err) < 0.000 else f"{std_err:.3f}"

            return f"{coef_str}{stars} ({std_err_str})"


        # Extract coefficients and significance of first stage
        first_stage_summary = first_stage_fit.summary2().tables[1]
        first_stage_coefs = first_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Extract coefficients and significance of second stage
        second_stage_summary = second_stage_fit.summary2().tables[1]
        second_stage_coefs = second_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Adding R-squared and F-statistic
        first_stage_coefs.loc["Adj. R-squared"] = f"{first_stage_fit.rsquared_adj:.3f}"
        first_stage_coefs.loc["F-statistic"] = f"{first_stage_fit.fvalue:.3f}"
        second_stage_coefs.loc["Adj. R-squared"] = f"{second_stage_fit.rsquared_adj:.3f}"
        
        # Adding Standard Errors
        second_stage_se = second_stage_fit.bse["predicted"]
        second_stage_coefs.loc["std err"] = second_stage_se
        
        # Residuals
        res = second_stage_fit.resid
        
        # Append 
        first_stage_results.append(first_stage_coefs)
        second_stage_results.append(second_stage_coefs)
        residuals.append(res)

    # Convert the results to DataFrame
    first_stage_df = pd.DataFrame(first_stage_results).T
    second_stage_df = pd.DataFrame(second_stage_results).T
    
    # Df of residuals
    residuals_df = pd.DataFrame(residuals).T

    return first_stage_df, second_stage_df, residuals_df

## Use the Function

In [11]:
# DataFrame that contains all the variables for the regression analysis
dep_var = merged_df[["log_distance_carbon_c", 
                     "s_industry",
                     "s_prot", 
                     "s_german", 
                     "longitud", 
                     "latitud", 
                     "soil_suitability", 
                     "mean_temp", 
                     "mean_preci", 
                     "dist_100km",
                     "uni_dummy", 
                     "min_distance_uni", 
                     "prov_cap_dummy", 
                     "BER_dummy", 
                     "density",
                     "steamengines_number",
                     "per_low",
                     "per_medium",
                     "per_unskilled",
                     "per_high",
                     "d_Hauptberuf",
                     "horsepower"]]

In [12]:
# Make two lists of exogenous variables
# Exclude latitude and longitude once
control_list = ["longitud",
                "latitud",
                "soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "BER_dummy"]

control_list_2 = ["soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "BER_dummy"]

In [13]:
first_unskilled, second_unskilled, res_unskilled = twoSLS(dep_var,
                                                          "log_distance_carbon_c",
                                                          "steamengines_number",
                                                          control_list,
                                                          "per_unskilled")

first_low, second_low, res_low = twoSLS(dep_var,
                                        "log_distance_carbon_c",
                                        "steamengines_number",
                                        control_list,
                                        "per_low")

first_medium, second_medium, res_medium = twoSLS(dep_var,
                                                 "log_distance_carbon_c",
                                                 "steamengines_number",
                                                 control_list,
                                                 "per_medium")

first_high, second_high, res_high = twoSLS(dep_var,
                                           "log_distance_carbon_c",
                                           "steamengines_number",
                                           control_list,
                                           "per_high")

In [14]:
first_unskilled[[0,6,8,12]]

,0,6,8,12
Intercept,18.498*** (1.035),-135.072*** (20.494),-127.366*** (22.789),-67.121** (28.545)
log_distance_carbon_c,-1.500*** (0.256),-2.522*** (0.354),-2.539*** (0.371),-2.580*** (0.372)
Adj. R-squared,0.055,0.169,0.170,0.199
F-statistic,34.325,17.725,14.077,11.947
longitud,NaN,0.551*** (0.158),0.473*** (0.175),0.392** (0.185)
latitud,NaN,2.076*** (0.388),1.996*** (0.467),0.873 (0.554)
soil_suitability,NaN,-1.626*** (0.380),-1.655*** (0.380),-1.561*** (0.390)
mean_temp,NaN,4.404*** (0.704),4.333*** (0.727),5.117*** (0.733)
mean_preci,NaN,0.143*** (0.046),0.148*** (0.048),0.200*** (0.049)
dist_100km,NaN,-0.177 (1.933),-0.465 (1.949),-0.441 (1.953)


In [15]:
# First Stage (same for every regression)
second_unskilled[[0,6,8,12]]

,0,6,8,12
Intercept,73.108*** (2.772),20.745 (23.788),36.235 (23.933),36.084 (30.588)
predicted,-1.783*** (0.212),-0.810*** (0.172),-0.589*** (0.176),-0.557*** (0.166)
Adj. R-squared,0.109,0.235,0.265,0.353
std err,0.211717,0.172017,0.17575,0.165543
longitud,NaN,0.007 (0.219),-0.616*** (0.237),-0.536** (0.234)
latitud,NaN,1.294*** (0.363),1.717*** (0.413),1.579*** (0.578)
soil_suitability,NaN,-1.249** (0.543),-0.996* (0.541),-0.372 (0.508)
mean_temp,NaN,-2.223* (1.197),-4.201*** (1.244),-3.622*** (1.279)
mean_preci,NaN,-0.113* (0.066),-0.197*** (0.070),-0.208*** (0.073)
dist_100km,NaN,4.441* (2.367),2.729 (2.335),0.605 (2.232)


In [16]:
second_low[[0,6,8,12]]

,0,6,8,12
Intercept,3.731** (1.821),82.706*** (15.493),66.673*** (15.764),48.557** (20.786)
predicted,1.525*** (0.139),0.475*** (0.112),0.425*** (0.116),0.410*** (0.112)
Adj. R-squared,0.172,0.301,0.313,0.356
std err,0.139044,0.112034,0.115761,0.112492
longitud,NaN,0.283** (0.143),0.521*** (0.156),0.496*** (0.159)
latitud,NaN,-1.850*** (0.237),-1.782*** (0.272),-1.381*** (0.393)
soil_suitability,NaN,1.161*** (0.354),1.143*** (0.356),0.861** (0.345)
mean_temp,NaN,1.786** (0.780),2.298*** (0.820),1.776** (0.869)
mean_preci,NaN,0.170*** (0.043),0.178*** (0.046),0.165*** (0.049)
dist_100km,NaN,-2.666* (1.542),-1.919 (1.538),-1.079 (1.517)


In [17]:
second_medium[[0,6,8,12]]

,0,6,8,12
Intercept,22.609*** (1.382),-3.820 (12.075),-2.875 (12.021),13.859 (15.462)
predicted,0.232** (0.106),0.309*** (0.087),0.147* (0.088),0.131 (0.084)
Adj. R-squared,0.007,0.116,0.168,0.258
std err,0.105543,0.087321,0.088276,0.083682
longitud,NaN,-0.279** (0.111),0.084 (0.119),0.035 (0.118)
latitud,NaN,0.554*** (0.184),0.082 (0.207),-0.162 (0.292)
soil_suitability,NaN,-0.005 (0.276),-0.230 (0.272),-0.531** (0.257)
mean_temp,NaN,0.388 (0.608),1.779*** (0.625),1.738*** (0.647)
mean_preci,NaN,-0.050 (0.034),0.022 (0.035),0.044 (0.037)
dist_100km,NaN,-1.515 (1.202),-0.611 (1.173),0.544 (1.128)


In [18]:
second_high[[0,6,8,12]]

,0,6,8,12
Intercept,0.552*** (0.130),0.369 (1.150),-0.032 (1.168),1.500 (1.452)
predicted,0.026*** (0.010),0.026*** (0.008),0.017** (0.009),0.016** (0.008)
Adj. R-squared,0.010,0.101,0.119,0.267
std err,0.009951,0.008316,0.008579,0.007857
longitud,NaN,-0.011 (0.011),0.012 (0.012),0.005 (0.011)
latitud,NaN,0.002 (0.018),-0.017 (0.020),-0.036 (0.027)
soil_suitability,NaN,0.094*** (0.026),0.083*** (0.026),0.042* (0.024)
mean_temp,NaN,0.049 (0.058),0.125** (0.061),0.108* (0.061)
mean_preci,NaN,-0.006** (0.003),-0.003 (0.003),-0.001 (0.003)
dist_100km,NaN,-0.260** (0.114),-0.200* (0.114),-0.070 (0.106)
